# Ollama 기본 튜토리얼

### 이 튜토리얼에서 배울 내용

- Ollama 설치 및 기본 사용법
- Python에서 Ollama 연결하기
- 기본적인 채팅 생성
- 스트리밍 응답 받기
- 대화 히스토리 관리하기

## 1. 환경 설정

Ollama를 사용하기 위해서는 먼저 Ollama를 설치해야 한다. 공식 웹사이트(https://ollama.ai)에서 다운로드할 수 있다.

### Linux 설치
```bash
curl -fsSL https://ollama.com/install.sh | sh
```

### 설치 확인

터미널에서 다음 명령어로 설치를 확인할 수 있다:
```bash
ollama --version
```

### 모델 다운로드

Llama2 모델을 다운로드한다:
```bash
ollama pull llama2
```

In [12]:
# 필요한 라이브러리 설치
#%pip install -q pydantic requests ollama

In [16]:
import requests
import json

## 2. Ollama 서버 연결 확인

In [17]:
# Ollama 서버 URL 설정
BASE_URL = "http://localhost:11434"

# 서버 상태 확인
response = requests.get(f"{BASE_URL}/api/tags")
print("서버 상태:", response.status_code)
print("\n사용 가능한 모델:")
print(json.dumps(response.json(), indent=2, ensure_ascii=False))

서버 상태: 200

사용 가능한 모델:
{
  "models": [
    {
      "name": "gpt-oss:20b",
      "model": "gpt-oss:20b",
      "modified_at": "2025-11-17T22:27:06.208945013+09:00",
      "size": 13793441244,
      "digest": "17052f91a42e97930aa6e28a6c6c06a983e6a58dbb00434885a0cf5313e376f7",
      "details": {
        "parent_model": "",
        "format": "gguf",
        "family": "gptoss",
        "families": [
          "gptoss"
        ],
        "parameter_size": "20.9B",
        "quantization_level": "MXFP4"
      }
    },
    {
      "name": "exaone3.5:7.8b",
      "model": "exaone3.5:7.8b",
      "modified_at": "2025-11-17T22:25:53.255724282+09:00",
      "size": 4770664857,
      "digest": "c7c4e3d1ca22fe9225f18b35eb719f67e2ca96a42e7fd17294a45b83ba8fbf03",
      "details": {
        "parent_model": "",
        "format": "gguf",
        "family": "exaone",
        "families": [
          "exaone"
        ],
        "parameter_size": "7.8B",
        "quantization_level": "Q4_K_M"
      }
    }
  ]

## 3. 첫 번째 채팅 생성

In [20]:
# 간단한 프롬프트로 텍스트 생성
def generate_text(prompt, model="gpt-oss:20b"):
    """
    주어진 프롬프트로 텍스트를 생성한다
    
    Args:
        prompt: 입력 프롬프트
        model: 사용할 모델 이름
    
    Returns:
        생성된 텍스트
    """
    url = f"{BASE_URL}/api/generate"
    
    # 요청 데이터 구성
    data = {
        "model": model,
        "prompt": prompt,
        "stream": False  # 스트리밍 비활성화
    }
    
    # API 호출
    response = requests.post(url, json=data)
    result = response.json()
    
    return result["response"]

# 테스트
prompt = "Python이란 무엇인가요? 한 문장으로 설명해주세요."
response = generate_text(prompt)
print(f"질문: {prompt}")
print(f"\n답변: {response}")

질문: Python이란 무엇인가요? 한 문장으로 설명해주세요.

답변: Python은 가독성 높은 문법과 풍부한 표준 라이브러리를 바탕으로 웹 개발, 데이터 과학, 인공지능 등 다양한 분야에서 폭넓게 활용되는 고급 프로그래밍 언어입니다.


## 4. 채팅 API 사용하기

In [21]:
def chat(messages, model="gpt-oss:20b"):
    """
    메시지 리스트로 채팅을 진행한다
    
    Args:
        messages: 메시지 리스트 (role과 content를 포함)
        model: 사용할 모델 이름
    
    Returns:
        어시스턴트의 응답
    """
    url = f"{BASE_URL}/api/chat"
    
    # 요청 데이터 구성
    data = {
        "model": model,
        "messages": messages,
        "stream": False
    }
    
    # API 호출
    response = requests.post(url, json=data)
    result = response.json()
    
    return result["message"]["content"]

# 단일 메시지 테스트
messages = [
    {"role": "user", "content": "인공지능이란 무엇인가요?"}
]

response = chat(messages)
print(f"사용자: {messages[0]['content']}")
print(f"\n어시스턴트: {response}")

사용자: 인공지능이란 무엇인가요?

어시스턴트: **인공지능(Artificial Intelligence, AI)** 은 인간의 지능을 모방하거나 확장하기 위해 컴퓨터 시스템이나 소프트웨어가 학습하고 추론하며 문제를 해결하도록 설계된 기술·방법론의 총칭입니다. 간단히 말해 “계산 기계가 ‘생각’하고 ‘배우는’ 능력”을 목표로 하는 학문 분야이며, 크게 아래와 같은 요소들을 포함합니다.

| 구분 | 핵심 내용 | 예시 |
|------|-----------|------|
| **학습(Learning)** | 데이터를 통해 패턴을 인식하고 경험을 축적 | 지도학습(이미지 분류), 비지도학습(클러스터링), 강화학습(게임 AI) |
| **추론(Inference)** | 학습된 모델을 이용해 새로운 상황에서 결론을 도출 | 질의응답, 예측, 자연어 처리 |
| **지식 표현(Knowledge Representation)** | 사실·규칙·관계를 기계가 이해할 수 있도록 구조화 | OWL, RDF, 논리 프로그래밍 |
| **문제 해결(Problem Solving)** | 목표를 달성하기 위한 전략 생성 | 최적화, 경로 탐색, 퍼즐 해결 |
| **인지(Perception)** | 센서나 입력을 통해 현실 세계를 인식 | 컴퓨터 비전, 음성 인식, 센서 융합 |

### 1. 역사적 배경
- **1940–50년대**: 튜링 테스트, 시뮬레이션과 계산 이론.
- **1956년**: 다트머스 회의에서 “인공지능”이라는 용어가 정식 등장.
- **1960–70년대**: 룰 기반 전문가 시스템, 로봇 공학.
- **1980년대**: 기계학습 알고리즘(결정트리, 인공신경망)의 등장.
- **1990–2000년대**: 서포트 벡터 머신, 베이즈 모델 등.
- **2000년대 이후**: 딥러닝(딥 뉴럴 네트워크)의 폭발적 성장, 대형 언어 모델(LLM)과 자가학습 모델.

### 2. 주요 분류
| 분류 | 특징 | 대표 알고리즘/모델 |
|------|------|-------

## 5. 시스템 메시지 사용하기

In [22]:
# 시스템 메시지를 포함한 채팅
messages = [
    {
        "role": "system", 
        "content": "당신은 친절한 선생님입니다. 항상 쉽고 명확하게 설명하며, 예시를 들어줍니다."
    },
    {
        "role": "user", 
        "content": "변수란 무엇인가요?"
    }
]

response = chat(messages)
print("=== 시스템 메시지 포함 예제 ===")
print(f"시스템 역할: {messages[0]['content']}")
print(f"\n질문: {messages[1]['content']}")
print(f"\n답변: {response}")

=== 시스템 메시지 포함 예제 ===
시스템 역할: 당신은 친절한 선생님입니다. 항상 쉽고 명확하게 설명하며, 예시를 들어줍니다.

질문: 변수란 무엇인가요?

답변: **변수(Variable)**란 “값이 바뀔 수 있는 상자”라고 생각하면 됩니다.  
컴퓨터가 기억할 수 있는 일종의 **저장 공간**이며, 그 공간에는 숫자, 글자, 문자열 등 어떤 값이라도 넣을 수 있습니다.

| 용어 | 비유 | 예시 |
|------|------|------|
| **이름** | 라벨 | `age`, `name`, `price` |
| **형식(타입)** | 상자의 크기 | 정수형(`int`), 문자열형(`string`) 등 |
| **값** | 그 안에 들어 있는 내용 | 25, "사과", 3.14 등 |

---

## 1. 변수의 역할

1. **데이터를 저장**: 프로그램이 실행되는 동안 필요한 정보를 보관합니다.  
2. **값을 바꿔서 사용**: 같은 변수에 다른 값을 다시 저장하면, 이후에 사용하는 부분이 그 새로운 값을 반영합니다.  
3. **코드 재사용**: 변수를 쓰면 같은 값을 반복해서 쓰지 않아도 되므로 코드가 깔끔해집니다.

---

## 2. 변수 선언과 초기화

> **선언** → 변수가 무엇인지 컴퓨터에게 알려주는 단계  
> **초기화** → 그 변수에 처음 들어갈 값을 주는 단계

### 예시 (Python)

```python
# 변수 선언 + 초기화
age = 25          # age라는 변수를 만들고 25를 넣음
name = "민수"     # name이라는 변수를 만들고 "민수"를 넣음
```

### 예시 (JavaScript)

```javascript
let score = 0;    // score라는 변수를 만들고 0으로 초기화
const name = "하늘"; // 이름은 바뀌지 않으니 const(상수) 사용
```

---

## 3. 변수의 값 바꾸기

```python
age = 25          # 초

## 6. 대화 히스토리 관리하기

In [23]:
# 대화 히스토리를 저장할 리스트
conversation = [
    {
        "role": "system",
        "content": "당신은 도움이 되는 AI 어시스턴트입니다."
    }
]

def chat_with_history(user_message):
    """
    대화 히스토리를 유지하며 채팅한다
    
    Args:
        user_message: 사용자 메시지
    
    Returns:
        어시스턴트 응답
    """
    # 사용자 메시지를 히스토리에 추가
    conversation.append({
        "role": "user",
        "content": user_message
    })
    
    # 전체 히스토리로 응답 생성
    response = chat(conversation)
    
    # 어시스턴트 응답을 히스토리에 추가
    conversation.append({
        "role": "assistant",
        "content": response
    })
    
    return response

# 연속된 대화 테스트
print("=== 대화 시작 ===")

# 첫 번째 질문
user_msg_1 = "제 이름은 김철수입니다."
response_1 = chat_with_history(user_msg_1)
print(f"\n사용자: {user_msg_1}")
print(f"어시스턴트: {response_1}")

# 두 번째 질문 (이전 대화 참조)
user_msg_2 = "제 이름이 무엇이었죠?"
response_2 = chat_with_history(user_msg_2)
print(f"\n사용자: {user_msg_2}")
print(f"어시스턴트: {response_2}")

# 전체 대화 히스토리 확인
print(f"\n총 메시지 수: {len(conversation)}개")

=== 대화 시작 ===

사용자: 제 이름은 김철수입니다.
어시스턴트: 안녕하세요, 김철수님! 무엇을 도와드릴까요?

사용자: 제 이름이 무엇이었죠?
어시스턴트: 당신의 이름은 **김철수**입니다.

총 메시지 수: 5개


## 7. 스트리밍 응답 받기

In [24]:
def chat_stream(messages, model="gpt-oss:20b"):
    """
    스트리밍 방식으로 채팅 응답을 받는다
    
    Args:
        messages: 메시지 리스트
        model: 사용할 모델 이름
    """
    url = f"{BASE_URL}/api/chat"
    
    # 스트리밍 활성화
    data = {
        "model": model,
        "messages": messages,
        "stream": True
    }
    
    # 스트리밍 요청
    response = requests.post(url, json=data, stream=True)
    
    # 전체 응답을 저장할 변수
    full_response = ""
    
    # 스트리밍 데이터 처리
    for line in response.iter_lines():
        body = json.loads(line)
        
        # 메시지 내용 추출
        content = body.get("message", {}).get("content", "")
        full_response += content
        
        # 실시간으로 출력
        print(content, end="", flush=True)
        
        # 완료 확인
        if body.get("done", False):
            break
    
    print()  # 줄바꿈
    return full_response

# 스트리밍 테스트
print("=== 스트리밍 응답 예제 ===")
messages = [
    {"role": "user", "content": "머신러닝에 대해 3문장으로 설명해주세요."}
]

print("\n어시스턴트: ", end="")
response = chat_stream(messages)

=== 스트리밍 응답 예제 ===

머신러닝은 데이터에서 패턴을 학습해 예측이나 결정을 자동으로 수행하는 인공지능의 한 분야입니다. 주어진 입력과 정답(라벨) 사이의 관계를 수학적 모델로 표현하고, 이를 반복적으로 최적화하여 새로운 데이터에 대한 예측 정확도를 높입니다. 이러한 과정은 이미지 인식, 자연어 처리, 금융 사기 탐지 등 다양한 산업에서 활용됩니다.


## 8. 생성 파라미터 조정하기

In [25]:
def chat_with_options(messages, temperature=0.7, top_p=0.9, model="gpt-oss:20b"):
    """
    파라미터를 조정하여 채팅한다
    
    Args:
        messages: 메시지 리스트
        temperature: 창의성 조절 (0.0 ~ 1.0)
        top_p: 샘플링 범위 조절 (0.0 ~ 1.0)
        model: 사용할 모델
    
    Returns:
        어시스턴트 응답
    """
    url = f"{BASE_URL}/api/chat"
    
    # 옵션을 포함한 요청 데이터
    data = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": {
            "temperature": temperature,
            "top_p": top_p
        }
    }
    
    response = requests.post(url, json=data)
    result = response.json()
    
    return result["message"]["content"]

# 다양한 temperature 값으로 테스트
prompt_msg = [{"role": "user", "content": "창의적인 스토리를 한 문장으로 만들어주세요."}]

print("=== Temperature 비교 ===")

# 낮은 temperature (더 결정적)
print("\n[Temperature = 0.3]")
response_low = chat_with_options(prompt_msg, temperature=0.3)
print(response_low)

# 높은 temperature (더 창의적)
print("\n[Temperature = 1.0]")
response_high = chat_with_options(prompt_msg, temperature=1.0)
print(response_high)

=== Temperature 비교 ===

[Temperature = 0.3]
When the city of clocks fell silent, a young dreamer discovered that the missing gears were the whispers of forgotten wishes, and by reassembling them, she rewrote the rhythm of time so that every heartbeat sang a new tomorrow.

[Temperature = 1.0]
When the old lighthouse keeper finally unlocked the moonlit glass box, he found his future self waving from the shoreline, begging him to stop digging for buried starlight.


## 9. 모델 정보 확인하기

In [26]:
def show_model_info(model_name="gpt-oss:20b"):
    """
    모델의 상세 정보를 출력한다
    
    Args:
        model_name: 조회할 모델 이름
    """
    url = f"{BASE_URL}/api/show"
    data = {"name": model_name}
    
    response = requests.post(url, json=data)
    info = response.json()
    
    print(f"=== {model_name} 모델 정보 ===")
    print(f"\n모델 세부사항:")
    print(json.dumps(info, indent=2, ensure_ascii=False))

# 모델 정보 확인
show_model_info()

=== gpt-oss:20b 모델 정보 ===

모델 세부사항:
{
  "license": "\n                                 Apache License\n                           Version 2.0, January 2004\n                        http://www.apache.org/licenses/\n\n   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION\n\n   1. Definitions.\n\n      \"License\" shall mean the terms and conditions for use, reproduction,\n      and distribution as defined by Sections 1 through 9 of this document.\n\n      \"Licensor\" shall mean the copyright owner or entity authorized by\n      the copyright owner that is granting the License.\n\n      \"Legal Entity\" shall mean the union of the acting entity and all\n      other entities that control, are controlled by, or are under common\n      control with that entity. For the purposes of this definition,\n      \"control\" means (i) the power, direct or indirect, to cause the\n      direction or management of such entity, whether by contract or\n      otherwise, or (ii) ownership of fift

## 10. 실용적인 채팅봇 예제

In [27]:
class SimpleChatbot:
    """
    Ollama를 사용하는 간단한 채팅봇 클래스다
    """
    
    def __init__(self, model="gpt-oss:20b", system_message=None):
        """
        채팅봇을 초기화한다    
        Args:
            model: 사용할 모델 이름
            system_message: 시스템 메시지 (선택사항)
        """
        self.model = model
        self.history = []
        
        # 시스템 메시지가 있으면 추가
        if system_message:
            self.history.append({
                "role": "system",
                "content": system_message
            })
    
    def send_message(self, user_message):
        """
        사용자 메시지를 보내고 응답을 받는다
        Args:
            user_message: 사용자 메시지
        
        Returns:
            어시스턴트 응답
        """
        # 사용자 메시지 추가
        self.history.append({
            "role": "user",
            "content": user_message
        })
        
        # 응답 생성
        response = chat(self.history, self.model)
        
        # 어시스턴트 응답 추가
        self.history.append({
            "role": "assistant",
            "content": response
        })
        
        return response
    
    def clear_history(self):
        """
        대화 히스토리를 초기화한다
        """
        # 시스템 메시지만 남기고 초기화
        system_messages = [msg for msg in self.history if msg["role"] == "system"]
        self.history = system_messages
    
    def get_history_length(self):
        """
        현재 대화 히스토리 길이를 반환한다
        """
        return len(self.history)

# 채팅봇 사용 예제
print("=== 채팅봇 예제 ===")

# 채팅봇 생성
bot = SimpleChatbot(
    system_message="당신은 친절하고 도움이 되는 AI 어시스턴트입니다."
)

# 대화 1
user_input_1 = "안녕하세요!"
response_1 = bot.send_message(user_input_1)
print(f"\n사용자: {user_input_1}")
print(f"봇: {response_1}")

# 대화 2
user_input_2 = "Python을 배우고 싶은데 어디서부터 시작하면 좋을까요?"
response_2 = bot.send_message(user_input_2)
print(f"\n사용자: {user_input_2}")
print(f"봇: {response_2}")

# 대화 3
user_input_3 = "추천해주신 방법으로 시작하겠습니다. 감사합니다!"
response_3 = bot.send_message(user_input_3)
print(f"\n사용자: {user_input_3}")
print(f"봇: {response_3}")

# 히스토리 확인
print(f"\n총 대화 메시지 수: {bot.get_history_length()}개")

=== 채팅봇 예제 ===

사용자: 안녕하세요!
봇: 안녕하세요! 어떻게 도와드릴까요? 😊

사용자: Python을 배우고 싶은데 어디서부터 시작하면 좋을까요?
봇: 안녕하세요!  
Python을 처음 배우는 길은 조금 긴 여정처럼 느껴질 수 있지만, 차근차근 단계별로 진행하면 재미있게 익힐 수 있어요. 😊  

아래는 **초보자에게 딱 맞는 “시작부터 실전까지” 로드맵**입니다. 필요에 따라 스킵하거나 바로 진행해도 좋습니다.

---

## 1️⃣ 준비 단계 – “Python 설치 & 개발 환경 만들기”

| 단계 | 내용 | 팁 |
|------|------|----|
| **① Python 설치** | - 공식 홈페이지 <https://www.python.org/downloads/> 에서 최신 버전(2025년 기준 3.12+) 다운로드<br>- Windows: “Add Python 3.x to PATH” 체크<br>- macOS: `brew install python` (Homebrew 필요)<br>- Linux: 대부분 배포판에 기본 설치, 혹은 `sudo apt install python3` | **Windows 사용자**는 Anaconda(데이터 과학용)도 설치하면 편리합니다. |
| **② 가상 환경 만들기** | ```bash<br>python -m venv myenv<br>source myenv/bin/activate  # macOS/Linux<br>myenv\Scripts\activate  # Windows<br>``` | 가상 환경을 쓰면 프로젝트마다 패키지 버전을 독립적으로 관리할 수 있어요. |
| **③ 코드 편집기/IDE** | - VS Code (가볍고 확장성 좋음)<br>- PyCharm (전문가용) | VS Code 설치 후 `Python` 확장과 `Pylance`를 설치하면 Linting·Auto‑Completion이 훌륭합니다. |
| **④ 기본 커맨드** | ```bash<br>python  # 인터프리터<br>pyt